In [4]:
## Dataset (Tips Dataset)
import seaborn as sns
df = sns.load_dataset('tips')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [6]:
df.columns

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')

In [7]:
## independent and dependent features
X = df[['tip', 'sex', 'smoker', 'day', 'time','size']]
Y = df['total_bill']

In [8]:
## Train test split
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.25,random_state=10)

In [9]:
X_train

,tip,sex,smoker,day,time,size
58,1.76,Male,Yes,Sat,Dinner,2
1,1.66,Male,No,Sun,Dinner,3
2,3.50,Male,No,Sun,Dinner,3
68,2.01,Male,No,Sat,Dinner,2
184,3.00,Male,Yes,Sun,Dinner,2
...,...,...,...,...,...,...
64,2.64,Male,No,Sat,Dinner,3
15,3.92,Male,No,Sun,Dinner,2
228,2.72,Male,No,Sat,Dinner,2
125,4.20,Female,No,Thur,Lunch,6


In [10]:
## We have two kind of value in category sex, smoker,time so we use (feature encoding [label encoding, onehot encoding])
## First doing label encoding for binary datas
from sklearn.preprocessing import LabelEncoder

In [11]:
le1 = LabelEncoder()
le2 = LabelEncoder()
le3 = LabelEncoder()

In [12]:
X_train['sex'] = le1.fit_transform(X_train['sex'])
X_train['smoker'] = le2.fit_transform(X_train['smoker'])
X_train['time'] = le3.fit_transform(X_train['time'])

In [13]:
X_train

,tip,sex,smoker,day,time,size
58,1.76,1,1,Sat,0,2
1,1.66,1,0,Sun,0,3
2,3.50,1,0,Sun,0,3
68,2.01,1,0,Sat,0,2
184,3.00,1,1,Sun,0,2
...,...,...,...,...,...,...
64,2.64,1,0,Sat,0,3
15,3.92,1,0,Sun,0,2
228,2.72,1,0,Sat,0,2
125,4.20,0,0,Thur,1,6


In [14]:
## same for test data
X_test['sex'] = le1.transform(X_test['sex'])
X_test['smoker'] = le2.transform(X_test['smoker'])
X_test['time'] = le3.transform(X_test['time'])

In [15]:
X_test

,tip,sex,smoker,day,time,size
162,2.00,0,0,Sun,0,3
60,3.21,1,1,Sat,0,2
61,2.00,1,1,Sat,0,2
63,3.76,1,1,Sat,0,4
69,2.09,1,1,Sat,0,2
...,...,...,...,...,...,...
201,2.01,0,1,Thur,1,2
149,2.00,1,0,Thur,1,2
175,3.11,1,1,Sun,0,2
226,2.00,0,1,Fri,1,2


In [16]:
## applying onehot encoding for multivalued category (columnTransformer)
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder


In [17]:
ct = ColumnTransformer(transformers=[(('onehot',OneHotEncoder(drop='first'),[3]))],remainder='passthrough')

In [18]:
X_train = ct.fit_transform(X_train)

In [19]:
X_test = ct.fit_transform(X_test)

In [20]:
## SVR  -- Support Vector Regression
from sklearn.svm import SVR
svr = SVR()

In [21]:
svr.fit(X_train,Y_train)
Y_pred = svr.predict(X_test)

In [22]:
## Performance metrices 
from sklearn.metrics import r2_score,mean_absolute_error
print(mean_absolute_error(Y_test,Y_pred))
print(r2_score(Y_test,Y_pred))

4.1486423210190235
0.46028114561159283


In [23]:
## HyperParameter tunning using GridSearchCV
from sklearn.model_selection import GridSearchCV
param_grid = {
    "C":[0.1,1,10,100,1000],
    "gamma":[1,0.1,0.01,0.001,0.0001],
    "kernel":['liner','rbf']
}


In [24]:
grid = GridSearchCV(SVR(),param_grid=param_grid,verbose=3,refit=True)
grid.fit(X_train,Y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=liner;, score=nan total time=   0.0s
[CV 2/5] END ........C=0.1, gamma=1, kernel=liner;, score=nan total time=   0.0s
[CV 3/5] END ........C=0.1, gamma=1, kernel=liner;, score=nan total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=liner;, score=nan total time=   0.0s
[CV 5/5] END ........C=0.1, gamma=1, kernel=liner;, score=nan total time=   0.0s
[CV 1/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.067 total time=   0.0s
[CV 2/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.058 total time=   0.0s
[CV 3/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.145 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.025 total time=   0.0s
[CV 5/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.089 total time=   0.0s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=liner;, score=nan total time=   0.0s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=lin

c:\StartingML\venv\lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
125 fits failed out of a total of 250.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
125 fits failed with the following error:
Traceback (most recent call last):
  File "c:\StartingML\venv\lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\StartingML\venv\lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "c:\StartingML\venv\lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "c:\StartingML\venv\lib\site-packages\sklearn\utils\_param_valida

GridSearchCV(estimator=SVR(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['liner', 'rbf']},
             verbose=3)

In [25]:
print(grid.best_score_)
print(grid.best_params_)

0.492086225466603
{'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}


In [26]:
grid_pred = grid.predict(X_test)

In [28]:
## Performance metrices 
from sklearn.metrics import r2_score,mean_absolute_error
print(mean_absolute_error(Y_test,grid_pred))
print(r2_score(Y_test,grid_pred))


3.8685147526100234
0.5081618245078687
